# Configuration

Funflow provides support for configuring a `Flow` via a YAML config file or environment variables. Support for automatically generated CLI flags is also planned but as of the writing of this tutorial has not been implemented.

If a Task you are using contains a `Configurable` field, you will need to specify the source for that configuration when you write your `Flow` via one of the `Literal`, `ConfigFromEnv`, or `ConfigFromFile` constructors. For example, the `Arg` field of the `DockerTask` supports configurable args. Let's take a look at a few examples.

## Environment Variables

In [12]:
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE GADTs #-}
-- Note: Using OverloadedStrings with DockerTask since it will automatically
-- make sure that any `Literal` strings we write are of type `Arg`
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.CAS.ContentStore as CS
import Funflow
import Funflow.Tasks.Docker
import Funflow.Config (Configurable (Literal, ConfigFromFile, ConfigFromEnv, ConfigFromCLI))

flow1 = dockerFlow $ 
    DockerTaskConfig {
        image="alpine:latest",
        command="echo",
        args=["this is a hard-coded literal value, the next value is: ", Arg $ ConfigFromEnv "CONFIGURING_FLOWS"]
    }

Now we just need to set the `CONFIGURING_FLOWS` environment variable and run the task

In [13]:
import System.Environment (setEnv)

setEnv "CONFIGURING_FLOWS" "'hello from an environment variable!'"

runFlow flow1 DockerTaskInput {inputBindings = [], argsVals = mempty} :: IO (CS.Item)

Found docker images, pulling...
Pulling docker image: alpine:latest
2020-11-18T08:31:20.630500935Z this is a hard-coded literal value, the next value is:  hello from an environment variable!
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}

## YAML File
For passing in configurations via a config file, use the `ConfigFromFile` constructor and pass a config file path to `runFlowWithConfig`:

In [14]:
{-# LANGUAGE QuasiQuotes #-}

import Path (reldir, relfile, (</>), parseAbsDir)
import System.Directory (getCurrentDirectory)
      
flow2 = dockerFlow $ 
    DockerTaskConfig {
        image="alpine:latest",
        command="echo",
        args=[Arg $ ConfigFromFile "ourMessage"]
    }
do
    cwd <- parseAbsDir =<< getCurrentDirectory
    let storeDirPath = cwd </> [reldir|./.tmp/store|]
        configFilePath = cwd </> [relfile|./flow.yaml|]
        flowConfig = RunFlowConfig {configFile = Just configFilePath, storePath = storeDirPath}
    runFlowWithConfig flowConfig flow2 DockerTaskInput {inputBindings = [], argsVals = mempty} :: IO CS.Item

Found docker images, pulling...
Pulling docker image: alpine:latest
2020-11-18T08:31:35.371344954Z Hello from the flow.yaml
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}